In [53]:
import torch
from torch import nn
from torch import autograd as ag
from torch.autograd import Variable
from torch.utils import data
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as pt
%matplotlib inline

In [44]:
class Discriminator(nn.Module):
    
    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv_layers= nn.Sequential(nn.Conv2d(1, 32, (3,3), padding=1),
                                 nn.ReLU(),
                                 nn.MaxPool2d((2,2)),
                                 nn.Conv2d(32, 64, (3,3), padding=1),
                                 nn.ReLU(),
                                 nn.MaxPool2d((2,2)))
        self.linear_layers = nn.Sequential(
                                 nn.Linear(3136, 100),
                                 nn.ReLU(),
                                 nn.Linear(100, 50),
                                 nn.ReLU(),
                                 nn.Linear(50, 2),
                                 nn.Softmax()
                                 # nn.MaxPool2d((2,2))
                                 )
    
    def forward(self, inp):
        conv_output = self.conv_layers(inp)
        return self.linear_layers(conv_output.view(conv_output.size(0), -1))

In [69]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            nn.ConvTranspose2d(1, 128, (5, 5)), # 14
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, (5, 5)), # 18
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, (5, 5)), # 22
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, (3, 3)), # 24
            nn.ReLU(),
            nn.ConvTranspose2d(16, 8, (3, 3)), # 26
            nn.ReLU(),
            nn.ConvTranspose2d(8, 1, (3, 3)), # 28
        )
    
    def forward(self, z):
        return self.net(z)

In [55]:
# inp = Variable(torch.rand( 1, 1, 28, 28))
d = Discriminator()
num_epochs = 1
train_loader = data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=transforms.Compose([
                       transforms.ToTensor()])))
test_loader = data.DataLoader(
    datasets.MNIST('../data', train=False, download=True, transform=transforms.Compose([
                       transforms.ToTensor()])))

count =0
for batch, (train_data, target) in enumerate(train_loader):
    if count == 1000:
        break
    count +=1
    # print(" count : ", count)
    # print("train : ", train_data)
    # print("target : ", target)
    for epoch in range(num_epochs):
        output = d.forward(Variable(train_data))
        optimizer = optim.Adam(d.parameters())
        # target = Variable(target)
        # target_t = torch.zeros(1, 10)
        # print(target_t[0][target-1])
        # target_t[0][target] = 1
        target_t = torch.Tensor([[1, 0]])
        # print("size ", target.size())
        loss_obj = nn.MSELoss()
        loss = loss_obj(output, Variable(target_t))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print(" epoch : ", epoch, "loss : ", loss.data[0])

In [72]:
g = Generator()
z = torch.rand(1,1,10,10)
gen_epochs = 100
# print(g.forward(Variable(z)))
# print(len(train_loader[0]))
count = 0 
for batch, (train_data, target) in enumerate(test_loader):
    if count >= 100:
        break
    count +=1
    for epoch in range(gen_epochs):
        ce_loss = nn.CrossEntropyLoss()
        d_loss = ce_loss(d.forward(Variable(train_data)), Variable(torch.LongTensor(1)))
        z = Variable(torch.rand(1,1,10,10))
        g_loss = ce_loss(d.forward(g.forward(z)), Variable(torch.LongTensor(1)))
        tot_loss = d_loss + g_loss
        optimizer_d = optim.SGD(d.parameters(), lr=1e-3)
        tot_loss.backward()
        optimizer_d.step()
        
        optimizer_g = optim.SGD(g.parameters(), lr=1e-3)
        g_loss = ce_loss(d.forward(g.forward(z)), Variable(torch.LongTensor(1)))
        g_loss.backward()
        optimizer_g.step()
        if epoch == 99:
            print(" batch : ", batch)
            print(" tot loss : ", tot_loss.data[0])

 batch :  0
 tot loss :  0.6265233755111694


 batch :  1
 tot loss :  0.6265233755111694


 batch :  2
 tot loss :  0.6265233755111694


 batch :  3
 tot loss :  0.6265233755111694


 batch :  4
 tot loss :  0.6265233755111694


 batch :  5
 tot loss :  0.6265233755111694


 batch :  6
 tot loss :  0.6265233755111694


 batch :  7
 tot loss :  0.6265233755111694


 batch :  8
 tot loss :  0.6265233755111694


KeyboardInterrupt: 